# This notebook shows example to load the CHGNet for prediction


In [ ]:
try:
    import chgnet
except ImportError:
    # install CHGNet (only needed on Google Colab or if you didn't install CHGNet yet)
    !pip install chgnet

In [ ]:
from pymatgen.core import Structure

from chgnet.model import CHGNet

### Read structure from a json or cif file


In [ ]:
try:
    from chgnet import ROOT

    structure = Structure.from_file(f"{ROOT}/examples/mp-18767-LiMnO2.cif")
except Exception:
    from urllib.request import urlopen

    url = "https://raw.githubusercontent.com/CederGroupHub/chgnet/main/examples/mp-18767-LiMnO2.cif"
    cif = urlopen(url).read().decode("utf-8")
    structure = Structure.from_str(cif, fmt="cif")

print(structure)

Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.868779   4.634475   5.832507
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Li+   0.5  0.5  0.37975
  1  Li+   0    0    0.62025
  2  Mn3+  0.5  0.5  0.863252
  3  Mn3+  0    0    0.136747
  4  O2-   0.5  0    0.360824
  5  O2-   0    0.5  0.098514
  6  O2-   0.5  0    0.901486
  7  O2-   0    0.5  0.639176


### Define Model


In [ ]:
chgnet = CHGNet.load()

# Alternatively you can read your own model
# chgnet = CHGNet.from_file(model_path)

CHGNet initialized with 400,438 parameters


# Predict energy, force, stress, magmom


In [ ]:
prediction = chgnet.predict_structure(structure)
for key, _unit in zip(
    ("energy", "forces", "stress", "magmom"), ("eV/atom", "eV/A", "GPa", "mu_B")
):
    print(f"CHGNet-predicted {key}={prediction[key[0]]}\n")

CHGNet-predicted energy=-7.371591567993164

CHGNet-predicted forces=[[ 4.4703484e-08 -4.2840838e-08  2.4071064e-02]
 [-4.4703484e-08 -1.4551915e-08 -2.4071217e-02]
 [-1.7881393e-07  1.0244548e-08  2.5402933e-02]
 [ 5.9604645e-08 -2.3283064e-08 -2.5402665e-02]
 [-1.1920929e-07  6.6356733e-08 -2.1660209e-02]
 [ 2.3543835e-06 -8.0077443e-06  9.5508099e-03]
 [-2.2947788e-06  7.9898164e-06 -9.5513463e-03]
 [-5.9604645e-08 -0.0000000e+00  2.1660626e-02]]

CHGNet-predicted stress=[[ 3.3677638e-01 -3.7330341e-07 -5.1117036e-06]
 [ 1.0701370e-06  2.4674933e-01  1.8477700e-05]
 [-4.4460303e-06  1.9265182e-05  4.0324528e-02]]

CHGNet-predicted magmom=[0.00521556 0.0052153  3.8572898  3.8572903  0.02538028 0.03706942
 0.03706949 0.02538028]



# Structure Optimization


In [ ]:
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()

CHGNet initialized with 400,438 parameters
CHGNet will run on cpu


In [ ]:
structure.perturb(0.1)
result = relaxer.relax(structure, verbose=False)
print(f"\nCHGNet took {len(result['trajectory'])} steps. Relaxed structure:")
print(result["final_structure"])


CHGNet took 29 steps. Relaxed structure:
Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.865864   4.648716   5.827764
angles:  89.917211  90.239405  89.975425
pbc   :       True       True       True
Sites (8)
  #  SP            a          b         c      magmom
---  ----  ---------  ---------  --------  ----------
  0  Li+    0.494018   0.479737  0.387171  0.00498427
  1  Li+    0.008464   0.006131  0.625817  0.00512926
  2  Mn3+   0.50073    0.502478  0.869608  3.85374
  3  Mn3+   0.997815  -0.000319  0.139344  3.859
  4  O2-    0.502142   0.009453  0.363411  0.0253105
  5  O2-    1.00293    0.502592  0.104559  0.0366638
  6  O2-    0.493749   0.998092  0.903592  0.0365367
  7  O2-   -0.002278   0.495108  0.645655  0.0248522


# Molecular Dynamics


In [ ]:
from chgnet.model.dynamics import MolecularDynamics

md = MolecularDynamics(
    atoms=structure,
    model=chgnet,
    ensemble="nvt",
    temperature=1000,  # in k
    timestep=2,  # in fs
    trajectory="md_out.traj",
    logfile="md_out.log",
    loginterval=100,
    use_device="cpu",  # use 'cuda' for faster MD, if not specified, CHGNet will automatically search for CUDA
)
md.run(50)  # run a 0.1 ps MD simulation

CHGNet will run on cpu


/Users/janosh/.venv/py310/lib/python3.10/site-packages/ase/md/nvtberendsen.py:95: RuntimeWarning: divide by zero encountered in scalar divide
  (self.temperature / old_temperature - 1.0) *


# Magmom Visualization


In [ ]:
supercell = structure.copy()
supercell.make_supercell([2, 2, 2])
print(supercell.composition)

Li+16 Mn3+16 O2-32


In [ ]:
import random

n_Li = int(supercell.composition["Li+"])
remove_ids = random.sample(list(range(n_Li)), n_Li // 2)

supercell.remove_sites(remove_ids)
print(supercell.composition)

Li+8 Mn3+16 O2-32


In [ ]:
result = relaxer.relax(supercell)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 19:33:07     -426.191837*      20.8815
FIRE:    1 19:33:07     -400.066006*     268.6776
FIRE:    2 19:33:07     -426.080700*      50.7814
FIRE:    3 19:33:08     -427.411331*       4.2803
FIRE:    4 19:33:08     -426.366394*      52.4735
FIRE:    5 19:33:08     -426.988091*      35.4947
FIRE:    6 19:33:09     -427.518917*       9.0208
FIRE:    7 19:33:09     -427.456432*      15.6711
FIRE:    8 19:33:09     -427.474777*      14.7084
FIRE:    9 19:33:09     -427.507435*      12.8331
FIRE:   10 19:33:10     -427.547436*      10.1189
FIRE:   11 19:33:10     -427.586182*       6.6958
FIRE:   12 19:33:11     -427.616276*       2.8986
FIRE:   13 19:33:11     -427.633926*       1.8915
FIRE:   14 19:33:12     -427.641483*       4.3156
FIRE:   15 19:33:12     -427.646664*       7.0416
FIRE:   16 19:33:13     -427.658947*       8.6467
FIRE:   17 19:33:14     -427.687492*       8.7184
FI

In [ ]:
import pandas as pd

df_magmom = pd.DataFrame({"Unrelaxed": chgnet.predict_structure(supercell)["m"]})
df_magmom["CHGNet relaxed"] = result["final_structure"].site_properties["magmom"]

In [ ]:
fig = df_magmom.hist(
    nbins=200,
    sharex=True,
    sharey=True,
    backend="plotly",
    barmode="overlay",
    layout={"title": "Magmom distribution"},
    opacity=0.7,
    range_x=[3, 4],
    template="plotly_white",
)
fig.layout.legend.update(title="", x=1, y=1, xanchor="right", yanchor="top")
fig.layout.xaxis.title = "Magnetic moment"
fig.show()